In [ ]:
!pip install np_utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for np_utils: filename=np_utils-0.6.0-py3-none-any.whl size=56438 sha256=bfa17c287b0c03d39daed82a7d43b834b2655ed1e5128ae8c8a275f32bc7f13c
  Stored in directory: /root/.cache/pip/wheels/b6/c7/50/2307607f44366dd021209f660045f8d51cb976514d30be7cc7
Successfully built np_utils


In [1]:
import numpy as np
import re
from keras.layers import Dense, LSTM, Input, Embedding, Dropout
from keras.models import Model, Sequential
from keras.optimizers import RMSprop
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.callbacks import LambdaCallback
import np_utils
from keras.models import load_model, save_model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt


ModuleNotFoundError: ignored

In [ ]:



load_saved_model = True
train_model = False

# Load in the text and perform some cleanup
seq_length = 20
filename = "/Users/norbert/Desktop/LSTM/data/rockyou-75.txt"

with open(filename, encoding='utf-8-sig') as f:
    text = f.read().strip()

# Convert text to characters
chars = sorted(list(set(text)))
total_chars = len(chars)
print(total_chars)

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))



# Vectorization
X = np.zeros((len(text) - seq_length, seq_length, total_chars), dtype=bool)
y = np.zeros((len(text) - seq_length, total_chars), dtype=bool)
for i in range(len(text) - seq_length):
    for t, char in enumerate(text[i:i + seq_length]):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[text[i + seq_length]]] = 1

if load_saved_model:
    model = load_model('/Users/norbert/Desktop/LSTM/models/model-saveLSTM-91-chars.h5')
else:
    model = Sequential()
    model.add(LSTM(256, input_shape=(seq_length, total_chars), return_sequences=True))  # Adjusted input shape
    model.add(LSTM(512, return_sequences=True))
    model.add(LSTM(512))
    model.add(Dropout(0.3))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(total_chars, activation='softmax'))

    optimizer = Adam(learning_rate=0.00001)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_text(length, diversity):
    start_index = np.random.randint(0, len(text) - seq_length - 1)
    generated = ''
    sentence = text[start_index: start_index + seq_length]
    generated += sentence
    for i in range(length):
        x_pred = np.zeros((1, seq_length, total_chars))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]
        sentence = sentence[1:] + next_char
        generated += next_char
    return generated

if train_model:
    history = model.fit(X, y, batch_size=32, epochs=100)
    model.save('/Users/norbert/Desktop/LSTM/models/model-saving-LSTM-91-chars.h5')
    # Plot training loss
    plt.plot(history.history['loss'])
    plt.title('Model Loss Over Epochs')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.show()


print(generate_text(200, 0.55))
with open('LSTM-output.txt', 'w') as file:
    # Write the generated text to the file
    file.write(generate_text(1000, 0.5))


In [ ]:
#Tutaj mozna chyba zaczac jezeli jest zaladowany model i chcemy "personalizowac"
def vectorize_text(text, seq_length, total_chars, char_indices):
    X = np.zeros((len(text) - seq_length, seq_length, total_chars), dtype=bool)
    y = np.zeros((len(text) - seq_length, total_chars), dtype=bool)
    for i in range(len(text) - seq_length):
        for t, char in enumerate(text[i:i + seq_length]):
            X[i, t, char_indices[char]] = 1
        y[i, char_indices[text[i + seq_length]]] = 1
    return X, y


In [ ]:
def read_words_from_file(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        words = [line.strip() for line in f if line.strip()]
    return words


def fine_tune_on_words(model, words, char_indices, indices_char, seq_length, total_chars, epochs=5):
    fine_tune_text = ' '.join(words)
    X_fine, y_fine = vectorize_text(fine_tune_text, seq_length, total_chars, char_indices)
    history = model.fit(X, y, batch_size=32, epochs=30)
    return model


In [ ]:
personalized_words = ['Brena', 'Brenda', 'Brenn', 'Brenna', 'Brett', 'Bria', 'Briana', 'Brianna', 'Brianne', 'Bride', 'Bridget', 'Bridgette', 'Bridie', 'Brier', 'Brietta', 'Brigid', 'Brigida', 'Brigit', 'Brigitta', 'Brigitte', 'Brina', 'Briney', 'Brinn', 'Brinna', 'Briny', 'Brit', 'Brita', 'Britney', 'Britni', 'Britt', 'Britta', 'Brittan', 'Brittaney', 'Brittani', 'Brittany', 'Britte', 'Britteny', 'Brittne', 'Brittney', 'Brittni', 'Brook', 'Brooke', 'Brooks', 'Brunhilda', 'Brunhilde', 'Bryana', 'Bryn', 'Bryna', 'Brynn', 'Brynna', 'Brynne', 'Buffy', 'Bunni', 'Bunnie', 'Bunny', 'Cacilia', 'Cacilie', 'Cahra', 'Cairistiona', 'Caitlin', 'Caitrin', 'Cal', 'Calida', 'Calla', 'Calley', 'Calli', 'Callida', 'Callie', 'Cally', 'Calypso', 'Cam', 'Camala', 'Camel', 'Camella', 'Camellia', 'Cami', 'Camila', 'Camile', 'Camilla', 'Camille', 'Cammi', 'Cammie', 'Cammy', 'Candace', 'Candi', 'Candice', 'Candida', 'Candide', 'Candie', 'Candis', 'Candra', 'Candy', 'Caprice', 'Cara', 'Caralie', 'Caren', 'Carena', 'Caresa', 'Caressa', 'Caresse', 'Carey', 'Cari', 'Caria', 'Carie', 'Caril', 'Carilyn', 'Carin', 'Carina', 'Carine', 'Cariotta', 'Carissa', 'Carita', 'Caritta', 'Carla', 'Carlee', 'Carleen', 'Carlen', 'Carlene', 'Carley', 'Carlie', 'Carlin', 'Carlina', 'Carline', 'Carlita', 'Carlota', 'Carlotta', 'Carly', 'Carlye', 'Carlyn', 'Carlynn', 'Carlynne', 'Carma', 'Carmel', 'Carmela', 'Carmelia', 'Carmelina', 'Carmelita', 'Carmella', 'Carmelle', 'Carmen', 'Carmencita', 'Carmina', 'Carmine', 'Carmita', 'Carmon', 'Caro', 'Carol', 'Carol-Jean', 'Carola', 'Carolan', 'Carolann', 'Carole', 'Carolee', 'Carolin', 'Carolina', 'Caroline', 'Caroljean', 'Carolyn', 'Carolyne', 'Carolynn', 'Caron', 'Carree', 'Carri', 'Carrie', 'Carrissa', 'Carroll', 'Carry', 'Cary', 'Caryl', 'Caryn', 'Casandra', 'Casey', 'Casi', 'Casie', 'Cass', 'Cassandra', 'Cassandre', 'Cassandry', 'Cassaundra', 'Cassey', 'Cassi', 'Cassie', 'Cassondra', 'Cassy', 'Catarina', 'Cate', 'Caterina', 'Catha', 'Catharina', 'Catharine', 'Cathe', 'Cathee', 'Catherin', 'Catherina', 'Catherine', 'Cathi', 'Cathie', 'Cathleen', 'Cathlene', 'Cathrin', 'Cathrine', 'Cathryn', 'Cathy', 'Cathyleen', 'Cati', 'Catie', 'Catina', 'Catlaina', 'Catlee', 'Catlin', 'Catrina', 'Catriona', 'Caty', 'Caye', 'Cayla', 'Cecelia', 'Cecil', 'Cecile', 'Ceciley', 'Cecilia', 'Cecilla', 'Cecily', 'Ceil', 'Cele', 'Celene', 'Celesta', 'Celeste', 'Celestia', 'Celestina', 'Celestine', 'Celestyn', 'Celestyna', 'Celia', 'Celie', 'Celina', 'Celinda', 'Celine', 'Celinka', 'Celisse', 'Celka', 'Celle', 'Cesya', 'Chad', 'Chanda', 'Chandal', 'Chandra', 'Channa', 'Chantal', 'Chantalle', 'Charil', 'Charin', 'Charis', 'Charissa', 'Charisse', 'Charita', 'Charity', 'Charla', 'Charlean', 'Charleen', 'Charlena', 'Charlene', 'Charline', 'Charlot', 'Charlotta', 'Charlotte', 'Charmain', 'Charmaine', 'Charmane', 'Charmian', 'Charmine', 'Charmion', 'Charo', 'Charyl', 'Chastity', 'Chelsae', 'Chelsea', 'Chelsey', 'Chelsie', 'Chelsy', 'Cher', 'Chere', 'Cherey', 'Cheri', 'Cherianne', 'Cherice', 'Cherida', 'Cherie', 'Cherilyn', 'Cherilynn', 'Cherin', 'Cherise', 'Cherish', 'Cherlyn', 'Cherri', 'Cherrita', 'Cherry', 'Chery', 'Cherye', 'Cheryl', 'Cheslie', 'Chiarra', 'Chickie', 'Chicky', 'Chiquia', 'Chiquita', 'Chlo', 'Chloe', 'Chloette', 'Chloris', 'Chris', 'Chrissie', 'Chrissy', 'Christa', 'Christabel', 'Christabella', 'Christal', 'Christalle', 'Christan', 'Christean', 'Christel', 'Christen', 'Christi', 'Christian', 'Christiana', 'Christiane', 'Christie', 'Christin', 'Christina', 'Christine', 'Christy', 'Christye', 'Christyna', 'Chrysa', 'Chrysler', 'Chrystal', 'Chryste', 'Chrystel', 'Cicely', 'Cicily', 'Ciel', 'Cilka', 'Cinda', 'Cindee', 'Cindelyn', 'Cinderella', 'Cindi', 'Cindie', 'Cindra', 'Cindy', 'Cinnamon', 'Cissiee', 'Cissy', 'Clair', 'Claire', 'Clara', 'Clarabelle', 'Clare', 'Claresta', 'Clareta', 'Claretta', 'Clarette', 'Clarey', 'Clari', 'Claribel', 'Clarice', 'Clarie', 'Clarinda', 'Clarine', 'Clarissa', 'Clarisse', 'Clarita', 'Clary', 'Claude', 'Claudelle', 'Claudetta', 'Claudette', 'Claudia', 'Claudie', 'Claudina', 'Claudine', 'Clea', 'Clem', 'Clemence', 'Clementia', 'Clementina', 'Clementine', 'Clemmie', 'Clemmy', 'Cleo', 'Cleopatra', 'Clerissa', 'Clio', 'Clo', 'Cloe', 'Cloris', 'Clotilda', 'Clovis', 'Codee', 'Codi', 'Codie', 'Cody', 'Coleen', 'Colene', 'Coletta', 'Colette', 'Colleen', 'Collen', 'Collete', 'Collette', 'Collie', 'Colline', 'Colly', 'Con', 'Concettina', 'Conchita', 'Concordia', 'Conni', 'Connie', 'Conny', 'Consolata', 'Constance', 'Constancia', 'Constancy', 'Constanta', 'Constantia', 'Constantina', 'Constantine', 'Consuela', 'Consuelo', 'Cookie', 'Cora', 'Corabel', 'Corabella', 'Corabelle', 'Coral', 'Coralie', 'Coraline', 'Coralyn', 'Cordelia', 'Cordelie', 'Cordey', 'Cordi', 'Cordie', 'Cordula', 'Cordy', 'Coreen', 'Corella', 'Corenda', 'Corene', 'Coretta', 'Corette', 'Corey', 'Cori', 'Corie', 'Corilla', 'Corina', 'Corine', 'Corinna', 'Corinne', 'Coriss', 'Corissa', 'Corliss', 'Corly', 'Cornela', 'Cornelia', 'Cornelle', 'Cornie', 'Corny', 'Correna', 'Correy', 'Corri', 'Corrianne', 'Corrie', 'Corrina', 'Corrine', 'Corrinne', 'Corry', 'Cortney', 'Cory', 'Cosetta', 'Cosette', 'Costanza', 'Courtenay', 'Courtnay', 'Courtney', 'Crin', 'Cris', 'Crissie', 'Crissy', 'Crista', 'Cristabel', 'Cristal', 'Cristen', 'Cristi', 'Cristie', 'Cristin', 'Cristina', 'Cristine', 'Cristionna', 'Cristy', 'Crysta', 'Crystal', 'Crystie', 'Cthrine', 'Cyb', 'Cybil', 'Cybill', 'Cymbre', 'Cynde', 'Cyndi', 'Cyndia', 'Cyndie', 'Cyndy', 'Cynthea', 'Cynthia', 'Cynthie', 'Cynthy', 'Dacey', 'Dacia']
model = load_model('/content/model-saveLSTM-91-chars.h5')
optimizer = Adam(learning_rate=0.00001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
#model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.00001), loss='your_loss_function')
history = fine_tune_on_words(model, personalized_words, char_indices, indices_char, seq_length, total_chars)


In [ ]:

generated_text = generate_text(1000, 0.5)
with open('LSTM-output-fine-tuning.txt', 'w') as file:
    file.write(generated_text)
